In [3]:
import pandas as pd

# Read all sheets from the Excel file
excel_file = r'Q:\sachuriga\CR_CA1_paper\tables/CRs_counting.xlsx'  # Replace with your file path
all_sheets = pd.read_excel(excel_file, sheet_name=None)

all_tables = []
# Access each sheet as a DataFrame
for sheet_name, df in all_sheets.items():
    all_tables.append(df)
    print(f"Sheet: {sheet_name}")
    print(df)
    print("\n")

Sheet: Sheet1
    CRs + (Fissure)  CRs - (Fissure)
0        130.803913        28.514212
1        170.354486        24.458087
2        125.779552        24.090475
3        154.825801        18.745987
4        198.681207        53.301816
5        144.809955        46.733048
6        173.187968        34.415378
7        154.474449        27.527229
8        170.458271        42.658611
9        176.902332        61.743749
10       219.091604        22.351530
11       278.881534        55.460677


Sheet: Sheet2
    CRs + (Blade)  CRs - (Blade)
0      356.918508      16.582465
1      205.123997      33.040921
2      372.466395      39.405539
3      187.101941       7.981101
4      293.783221      54.754517
5      287.061500      39.839685
6      301.590387      29.867865
7      332.800370      42.213686
8      286.497037      68.214065
9      245.338567     107.648420
10     384.587807      67.093509
11     353.920799     119.289037


Sheet: Sheet3
    CRs + (Fissure and blade)  CRs - (Fissur

In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import shapiro, ttest_rel, wilcoxon
import warnings
warnings.filterwarnings('ignore')

# Ensure Matplotlib uses math rendering for superscripts
plt.rc('text', usetex=False)  # Use Mathtext (default); set to True if LaTeX is installed

excel_file = r'Q:\sachuriga\CR_CA1_paper\tables/CRs_counting.xlsx'  # Replace with your file path
all_sheets = pd.read_excel(excel_file, sheet_name=None)
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['svg.fonttype'] = 'none'
matplotlib.rcParams['font.family'] = 'Arial'

# Ensure there are exactly 3 sheets
if len(all_sheets) != 3:
    raise ValueError("Excel file must contain exactly 3 sheets")

# Set up the figure for 3 subplots with black background
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)
axes = axes.flatten()
fig.patch.set_facecolor('none')  # Transparent figure background

# Process each sheet
for idx, (sheet_name, df) in enumerate(all_sheets.items()):
    # Verify the DataFrame has exactly 2 columns
    if df.shape[1] != 2:
        raise ValueError(f"Sheet {sheet_name} must have exactly 2 columns")
    
    col1, col2 = df.columns
    data = df.melt(var_name='Column', value_name='Value')
    data['Index'] = np.repeat(df.index, 2)

    # Normality test (Shapiro-Wilk)
    stat1, p1 = shapiro(df[col1])
    stat2, p2 = shapiro(df[col2])
    normality = (p1 > 0.05) and (p2 > 0.05)  # Assume normal if p > 0.05

    # Statistical test
    if normality:
        stat, p_value = ttest_rel(df[col1], df[col2])
        test_name = 'Paired t-test'
    else:
        stat, p_value = wilcoxon(df[col1], df[col2])
        test_name = 'Wilcoxon signed-rank test'

    # Print comparison results
    print(f"\nSheet: {sheet_name}")
    print(f"Normality test (Shapiro-Wilk):")
    print(f"  {col1}: p-value = {p1:.4f} {'(Normal)' if p1 > 0.05 else '(Not Normal)'}")
    print(f"  {col2}: p-value = {p2:.4f} {'(Normal)' if p2 > 0.05 else '(Not Normal)'}")
    print(f"{test_name}: statistic = {stat:.4f}, p-value = {p_value:.4f}")
    print(f"Conclusion: {'Significant difference' if p_value < 0.05 else 'No significant difference'} (p < 0.05)")

    # Plot bar graph with SEM error bars
    bar_plot = sns.barplot(
        x='Column', 
        y='Value', 
        data=data, 
        ax=axes[idx], 
        errorbar='se',  # Use standard error of the mean for error bars
        capsize=0.2, 
        edgecolor=None,  # Grey edges for all bars
        linewidth=2, 
        errcolor='black',  # White error bars
        errwidth=2
    )

    # Set bar colors: blue for control, cyan for exp
    for bar, column in zip(bar_plot.patches, data['Column'].unique()):
        if 'crs +' in column.lower():
            bar.set_facecolor('blue')
        elif 'crs -' in column.lower():
            bar.set_facecolor('red')

    sns.stripplot(x='Column', y='Value', data=data, ax=axes[idx], color='black', size=5, jitter=True)
    
    # Replace all 'color="white"' with 'color="black"' throughout the plotting loop:
    axes[idx].set_xlabel('Column', color='black', fontsize=24).set_visible(False)
    axes[idx].set_ylabel(r'Density ($mm^{2}$)', color='black', fontsize=24)
    axes[idx].tick_params(axis='x', colors='black', labelsize=16, rotation=-15)
    axes[idx].tick_params(axis='y', colors='black')
    axes[idx].spines['left'].set_color('black')
    axes[idx].spines['bottom'].set_color('black')
    axes[idx].set_facecolor('none')  # Transparent axes background
    axes[idx].spines['top'].set_visible(False)
    axes[idx].spines['right'].set_visible(False)
    axes[idx].set_facecolor('none')  # Transparent axes background
# Save as transparent PDF
plt.tight_layout()
fig.savefig(r"counting.pdf", format="pdf", bbox_inches='tight')
plt.close()


Sheet: Sheet1
Normality test (Shapiro-Wilk):
  CRs + (Fissure): p-value = 0.0953 (Normal)
  CRs - (Fissure): p-value = 0.1982 (Normal)
Paired t-test: statistic = 12.6294, p-value = 0.0000
Conclusion: Significant difference (p < 0.05)

Sheet: Sheet2
Normality test (Shapiro-Wilk):
  CRs + (Blade): p-value = 0.4991 (Normal)
  CRs - (Blade): p-value = 0.2333 (Normal)
Paired t-test: statistic = 13.1965, p-value = 0.0000
Conclusion: Significant difference (p < 0.05)

Sheet: Sheet3
Normality test (Shapiro-Wilk):
  CRs + (Fissure and blade): p-value = 0.0253 (Not Normal)
  CRs - (Fissure and blade): p-value = 0.3857 (Normal)
Wilcoxon signed-rank test: statistic = 0.0000, p-value = 0.0005
Conclusion: Significant difference (p < 0.05)
